In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/MIMIC_IV_Pipeline/radiology.csv.gz', compression='gzip',nrows=200)
df.tail()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
195,10001338-RR-37,10001338,NaN,RR,37,2141-02-26 08:33:00,2141-02-26 10:50:00,CLINICAL HISTORY: Pain in the left upper oute...
196,10001338-RR-38,10001338,NaN,RR,38,2141-02-26 08:33:00,2141-02-26 10:29:00,CLINICAL HISTORY: Pain in the left upper oute...
197,10001338-RR-39,10001338,29335220.0,RR,39,2141-07-07 09:42:00,2141-07-07 10:52:00,INDICATION: Chest pressure. Please evaluate ...
198,10001338-RR-40,10001338,27987619.0,RR,40,2142-02-27 05:44:00,2142-02-27 07:52:00,HISTORY: ___ female with right lower quadrant...
199,10001338-RR-41,10001338,27987619.0,RR,41,2142-02-27 07:48:00,2142-02-27 09:25:00,INDICATION: ___ woman with right lower quadra...


In [3]:
!pip -q install -U transformers torch accelerate --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.


In [4]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

In [5]:
TEXT_COL = "text"
NOTE_ID_COL = "note_id"

df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str)

In [6]:
#using the model https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT

#MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
MODEL_NAME = "answerdotai/ModernBERT-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

ModernBertModel(
  (embeddings): ModernBertEmbeddings(
    (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (layers): ModuleList(
    (0): ModernBertEncoderLayer(
      (attn_norm): Identity()
      (attn): ModernBertAttention(
        (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
        (rotary_emb): ModernBertRotaryEmbedding()
        (Wo): Linear(in_features=768, out_features=768, bias=False)
        (out_drop): Identity()
      )
      (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): ModernBertMLP(
        (Wi): Linear(in_features=768, out_features=2304, bias=False)
        (act): GELUActivation()
        (drop): Dropout(p=0.0, inplace=False)
        (Wo): Linear(in_features=1152, out_features=768, bias=False)
      )
    )
    (1-21): 21 x ModernBertEncoderLayer(
      (attn_norm): LayerNorm((768,), eps=1e-05, e

In [7]:
def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

def embed_text(text, max_len=512, stride=64):
    #usse the long texts and chunking them --> caluculating the avg of the embeddings
    enc = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    chunk_embs = []
    for i in range(len(enc["input_ids"])):
        input_ids = enc["input_ids"][i].unsqueeze(0).to(device)
        attn_mask = enc["attention_mask"][i].unsqueeze(0).to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids, attention_mask=attn_mask)
            emb = mean_pool(out.last_hidden_state, attn_mask)  # (1,768)
            chunk_embs.append(emb[0])

    note_emb = torch.stack(chunk_embs).mean(dim=0)  # average chunks
    note_emb = torch.nn.functional.normalize(note_emb, p=2, dim=0)  # normalize
    return note_emb.cpu().tolist()

#run the embed func through the texts
all_embs = []
for text in tqdm(df[TEXT_COL].tolist(), desc="Embedding notes"):
    all_embs.append(embed_text(text))

df["text_embed"] = all_embs

Embedding notes: 100%|██████████| 200/200 [04:48<00:00,  1.44s/it]


In [8]:
out_path = "/content/drive/MyDrive/MIMIC_IV_Pipeline/Multimodal_Embeddings/results/notes_radiology_with_modernbert_embedding.csv"
df.to_csv(out_path, index=False)
print(f"Saved embeddings → {out_path}")
print(df[[NOTE_ID_COL, "text_embed"]].head(2))

Saved embeddings → /content/drive/MyDrive/MIMIC_IV_Pipeline/Multimodal_Embeddings/results/notes_radiology_with_modernbert_embedding.csv
          note_id                                         text_embed
0  10000032-RR-14  [-0.006691996939480305, -0.02236788161098957, ...
1  10000032-RR-15  [-0.018437141552567482, -0.024756865575909615,...


In [9]:
df.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,text_embed
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,"[-0.006691996939480305, -0.02236788161098957, ..."
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,"[-0.018437141552567482, -0.024756865575909615,..."
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi...","[-0.020959103479981422, -0.018208401277661324,..."
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...,"[-0.013200057670474052, -0.023104406893253326,..."
4,10000032-RR-20,10000032,NaN,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...,"[-0.013573058880865574, -0.023852651938796043,..."
